In [12]:
# -*- coding: utf-8 -*-
"""
IT Helpdesk Chatbot
A rule-based chatbot to handle common corporate IT support requests.
"""

import re
import random

# A simple in-memory database for tickets and conversation state.
tickets = {}
conversation_context = {}

# --- Entity Extraction Functions ---

def extract_ticket_id(user_input):
    """Extracts a ticket ID (e.g., TICKET-123) from the user's message."""
    match = re.search(r'\b(TICKET-\d{3})\b', user_input, re.IGNORECASE)
    return match.group(0).upper() if match else None

def extract_software_name(user_input):
    """Extracts a software name from a request."""
    # This is a simple regex that looks for capitalized words after keywords.
    match = re.search(r'\b(install|request|need)\s+([A-Z][a-zA-Z0-9\s\.]+)\b', user_input)
    return match.group(2).strip() if match else None

# --- Conversation Handler Functions ---

def handle_software_request(user_id, user_input):
    """Manages the multi-step process of creating a software request ticket."""
    context = conversation_context.get(user_id, {})

    if not context:
        software = extract_software_name(user_input)
        conversation_context[user_id] = {'intent': 'request_software', 'stage': 'ask_user', 'software': software}
        if software:
            return "Sure, I can help with that. What is your corporate username?"
        else:
            conversation_context[user_id]['stage'] = 'ask_software'
            return "Of course. What software do you need to install?"

    stage = context.get('stage')

    if stage == 'ask_software':
        context['software'] = user_input.strip()
        context['stage'] = 'ask_user'
        return "Got it. And what is your corporate username?"

    if stage == 'ask_user':
        username = user_input.strip()
        software = context['software']
        ticket_id = f"TICKET-{random.randint(100, 999)}"

        tickets[ticket_id] = {
            'username': username,
            'request': f"Install {software}",
            'status': 'Pending Approval'
        }
        # Clear context after completion
        del conversation_context[user_id]
        return f"Thank you, {username}. I have created ticket {ticket_id} for your request to install {software}. You can check its status anytime."

    return "Sorry, I got a bit lost. Let's try that again."

# --- Main Chatbot Logic ---

def get_chatbot_response(user_id, user_input):
    """The main router function that determines intent and generates a response."""
    # First, check if we are in the middle of a conversation.
    if user_id in conversation_context:
        if 'cancel' in user_input.lower():
            del conversation_context[user_id]
            return "Okay, I've cancelled the current request."
        return handle_software_request(user_id, user_input)

    # If not in a conversation, determine intent from the new input.
    user_input_lower = user_input.lower()

    if re.search(r'\b(hello|hi|hey)\b', user_input_lower):
        return "Hello! Welcome to the IT Helpdesk. How can I assist you today? (e.g., password reset, software request, ticket status)"

    if re.search(r'\b(password|reset|forgot|locked)\b', user_input_lower):
        return "To reset your password, please visit our self-service portal at https://passwordreset.mycompany.com"

    if re.search(r'\b(install|software|request|need)\b', user_input_lower):
        return handle_software_request(user_id, user_input)

    if re.search(r'\b(status|ticket|check)\b', user_input_lower):
        ticket_id = extract_ticket_id(user_input)
        if ticket_id:
            if ticket_id in tickets:
                status = tickets[ticket_id]['status']
                return f"The status for ticket {ticket_id} is: '{status}'."
            else:
                return f"Sorry, I could not find ticket {ticket_id} in our system."
        else:
            return "Please provide a ticket ID (e.g., 'What is the status of TICKET-123?') to check its status."

    if re.search(r'\b(bye|thanks|thank you)\b', user_input_lower):
        return "You're welcome! Goodbye."

    return "I'm sorry, I don't understand that. You can ask me to 'reset a password', 'request software', or 'check ticket status'."

# --- Main Interaction Loop ---

def main():
    """The main function to run the chatbot interaction loop."""
    print("=" * 40)
    print("    IT Helpdesk Bot Initialized")
    print("=" * 40)
    print("Type 'quit' to exit.")
    print("-" * 40)

    user_id = 'employee_01'  # Simulate a single user session

    while True:
        user_input = input("You: ")
        if user_input.lower() == 'quit':
            print("Bot: Session ended. Goodbye!")
            break

        response = get_chatbot_response(user_id, user_input)
        print(f"Bot: {response}")

if __name__ == "__main__":
    main()

    IT Helpdesk Bot Initialized
Type 'quit' to exit.
----------------------------------------
You: hi
Bot: Hello! Welcome to the IT Helpdesk. How can I assist you today? (e.g., password reset, software request, ticket status)
You: I'm locked out of my account
Bot: To reset your password, please visit our self-service portal at https://passwordreset.mycompany.com
You: ok thanks. I also need to install Photoshop
Bot: Sure, I can help with that. What is your corporate username?
You: Sakshi
Bot: Thank you, Sakshi. I have created ticket TICKET-636 for your request to install Photoshop. You can check its status anytime.
You: What is the status of Ticket-636
Bot: The status for ticket TICKET-636 is: 'Pending Approval'.
You: thanks bye
Bot: You're welcome! Goodbye.
You: quit
Bot: Session ended. Goodbye!
